In [ ]:
import pandas as pd
data = {"file_name":["test1.jpg","test2.jpg","test3.jpg"],
              "content":["Payment made to cashier in $2000.00", "Transaction was done using cash invoice#' 12390","Used debit card for payment in costco"],
           "page_label":["para","mil","bil"]}
read_res = pd.DataFrame(data)

In [ ]:
for ind in read_res.index:
  print(ind)

0
1
2


In [ ]:
read_res['file_name'][0]

'test1.jpg'

In [ ]:
files_list = ['file1.txt', 'test2.jpg', 'test3.jpg']

In [ ]:
read_res

,file_name,content,page_label
0,test1.jpg,Payment made to cashier in $2000.00,para
1,test2.jpg,Transaction was done using cash invoice#' 12390,mil
2,test3.jpg,Used debit card for payment in costco,bil


In [ ]:
def _if_repeat_num(segment:str):
        regex = '([1-9]+)\\1\\1\\1+'
        search_res = re.search(regex, segment)
        return True if search_res else False

def _if_single_num(segment:str):
    return True if segment.isnumeric() and len(segment)==1 else False

def _if_consecutive_num(segment:str):
    # Get all 2 or more consecutive Numbers
    lstNumbers = re.findall(r'\d{4}', segment)
    # Sort and convert list to Set to avoid duplicates
    setValues = set(sorted(lstNumbers))
    lstValues = list(setValues)
    for num in lstValues:
        if (str(num) in '0123456789'):
            return True

    return False

In [ ]:
def invoice_num_check(read_res):
        read_res['invalid_invoice_nbr'] = False
        read_res['possible_invoice_nbr'] = None
        invoice_num_keywords = ['invoice number', 'invoice #', 'invoice num', 'invoice#']
        for row in read_res.iterrows():
            # print(row[1]['file_name'])
            # locate keyword 'invoice'
            text = row[1]['content'].lower()
            start = 0

            for keyword in invoice_num_keywords:
                idx = text.find(keyword)
                print(idx)
                if idx == -1:
                  print("I am in")
                  continue
                print("continue")
                potential_invoice_num = text[idx: idx+25] if idx+25 < len(text) else text[idx:]
                print("entered")
                print(potential_invoice_num)
                start = idx+1
                # print(potential_invoice_num)
                if potential_invoice_num.find('date')>-1:
                    potential_invoice_num = potential_invoice_num[:potential_invoice_num.find('date')]

                print(potential_invoice_num)
                text_list = potential_invoice_num.split(' ')
                print(text_list)
                for segment in text_list:
                    if any(c.isdigit() for c in segment):
                        if  _if_single_num(segment):
                            # print('suspicious single invoice number: ' + segment)
                            read_res.loc[row[0], 'invalid_invoice_nbr'] = True
                            read_res.loc[row[0], 'possible_invoice_nbr'] = segment
                            break
                        elif _if_repeat_num(segment):
                            # print('suspicious repeat invoice number: ' + segment)
                            read_res.loc[row[0], 'invalid_invoice_nbr'] = True
                            read_res.loc[row[0], 'possible_invoice_nbr'] = segment
                            break
                        elif _if_consecutive_num(segment):
                            # print('suspicious consecutive invoice number: ' + segment)
                            read_res.loc[row[0], 'invalid_invoice_nbr'] = True
                            read_res.loc[row[0], 'possible_invoice_nbr'] = segment
                            break
                        else:
                            read_res.loc[row[0], 'possible_invoice_nbr'] = segment
                            break
        return read_res
res = invoice_num_check(read_res)
res

-1
I am in
-1
I am in
-1
I am in
-1
I am in
-1
I am in
-1
I am in
-1
I am in
32
continue
entered
invoice#' 12390
invoice#' 12390
["invoice#'", '12390']
-1
I am in
-1
I am in
-1
I am in
-1
I am in


,file_name,content,page_label,invalid_invoice_nbr,possible_invoice_nbr
0,test1.jpg,Payment made to cashier in $2000.00,para,False,None
1,test2.jpg,Transaction was done using cash invoice#' 12390,mil,False,12390
2,test3.jpg,Used debit card for payment in costco,bil,False,None


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=e00b25ccab121aeee0755a4876fc7dae8035b87bc23f701478a603c9b119cc25
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit,udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType, StringType, FloatType,DoubleType
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, col, expr, lower, split, udf
from pyspark.sql.functions import col, lower
import re
sc = SparkSession.builder.appName('example_spark').getOrCreate()

In [ ]:
# creating a dataframe
data = [
    ("test1.jpg", "Payment made to cashier with cash of 2000.00 invoice num 36241", "para",30.32),
    ("test2.jpg", "Transaction was done using cash in costco invoice#12534", "mil",10.2),
    ("test3.jpg","User with register no: 58290 used debit card for payment","bil",27.9)
]
headers = ("file_name", "content", "page_label","hand")
df = sc.createDataFrame(data, headers)

# paid_by_cash

In [ ]:
def paid_by_cash(read_res):
  cash_payment_invoice = []
  keywords = ['cash']
  payment_keywords = ['cash', 'money order', 'credit', 'visa', 'amex', 'debit', 'cheque', 'charge', 'mc', 'db', 'check', 'credit card', 'refund', 'roa', 'grant', 'master', 'cq']

  read_res = read_res.withColumn("paid_by_cash", lower(col("content")).contains("cash") & ~(lower(col("content")).contains("cashier")))

  #read_res = read_res.withColumn("pay_method", sum([col("content").contains(payment) for payment in payment_keywords]))
  #read_res = read_res.withColumn("paid_by_cash", col("paid_by_cash") & (col("pay_method") < 2))
  #read_res = read_res.drop("pay_method")

  return read_res

# has_dollar_symbol

In [ ]:
def has_dollar_symbol(self, read_res):
  def check_no_dollar_symbol(content):
      return '$' not in content

  check_no_dollar_symbol_udf = udf(check_no_dollar_symbol, BooleanType())
  read_res = read_res.withColumn('no_dollar_symbol', check_no_dollar_symbol_udf('content'))

  return read_res

# handwritten_check

In [ ]:
def handwritten_check(read_res, threshold):
		read_res = read_res.withColumn('above_handwritten_threshold', F.lit(False))
		#check_threshold = F.udf(lambda percentage: percentage > threshold, returnType=BooleanType())
		def check_threshold(hand,threshold):
		  return hand>threshold
		#check_threshold = F.udf(lambda percentage: int(percentage > threshold), returnType=IntegerType())
		#read_res = read_res.withColumn('hand', read_res['hand'].cast('float'))
		read_res = read_res.withColumn('above_handwritten_threshold', check_threshold(read_res['hand'],threshold))
		return read_res

# get_payment

In [ ]:
def get_payment(read_res):
		def extract_amount(text):
			potential_amount = re.findall(r'[$][\d{1,3},?]*\d{1,3}\.\d{2}', text)
			if potential_amount:
				amount_list = [float(i[1:].replace(",", "")) for i in potential_amount]
				return max(amount_list)
			potential_amount_2 = re.findall(r'[\d{1,3},?]*\d{1,3}\.\d{2}', text)
			if potential_amount_2:
				amount_list = [float(i.replace(",", "")) for i in potential_amount_2]
				return max(amount_list)
			return None

		extract_amount_udf = F.udf(extract_amount, DoubleType())
		read_res = read_res.withColumn('content', F.lower(F.col('content')))
		read_res = read_res.withColumn('payment_amount', extract_amount_udf(F.col('content')))
		return read_res

# get_reg_num

In [ ]:
def get_reg_num(read_res):
        read_res = read_res.withColumn("register_num", lit(None))
        invoice_num_keywords = ['phn/reg', 'reg#', 'reg #', 'register#', 'register #', 'register number', 'reg no', 'register no']
        Reg = "No Register Num"
        def extract_register_num(text):
            text = text.lower()
            text = text.replace('.', ' ')
            text = text.replace(',', ' ')
            text = text.replace(':', ' ')

            for keyword in invoice_num_keywords:
                idx = text.find(keyword)
                if idx == -1:
                    continue
                potential_invoice_num = text[idx: idx+25] if idx+25 < len(text) else text[idx:]

                text_list = potential_invoice_num.split(' ')
                for segment in text_list:
                    if any(c.isdigit() for c in segment):
                        segment = segment.replace('#', '')
                        segment = segment.replace(' ', '')
                        return segment
            return Reg
        extract_register_num_udf = udf(extract_register_num, StringType())

        read_res = read_res.withColumn("register_num", extract_register_num_udf(col("content")))

        return read_res

#  repeat num

In [ ]:
def if_repeat_num(segment):
        regex = r'([1-9]+)\1\1\1+'
        search_res = re.search(regex,segment)
        return True if search_res else False

#if_repeat_num_udf = udf(if_repeat_num, BooleanType())
#df7 = df.withColumn("has_rpeat_num",if_repeat_num_udf(col("segment")) )
#df7.show()

def if_single_num(segment):
  if segment.isnumeric() and len(segment)==1:
    return True
  else:
    return False
  #return True if segment.isnumeric() and len(segment)==1 else False
    #df8 = df.withColumn("Singlet_num",_if_single_num(col("segment")) )
    #df8.show()

def if_consecutive_num(segment):
        # Get all 2 or more consecutive Numbers
        lstNumbers = re.findall(r'\d{4}', segment)
        # Sort and convert list to Set to avoid duplicates
        setValues = set(sorted(lstNumbers))
        lstValues = list(setValues)
        for num in lstValues:
            if (str(num) in '0123456789'):
                return True

        return False

# invoice_num_check

In [ ]:
# def invoice_num_check(read_res):
#         invalid_invoice_nbr_udf = udf(lambda segment: if_single_num(segment) or if_repeat_num(segment) or if_consecutive_num(segment), BooleanType())
#         possible_invoice_nbr_udf = udf(lambda segment: segment if (if_single_num(segment) or if_repeat_num(segment) or if_consecutive_num(segment)) else None, StringType())

#         invoice_num_keywords = ['invoice number', 'invoice #', 'invoice num', 'invoice#']
#         read_res = read_res.withColumn('invalid_invoice_nbr', col('invalid_invoice_nbr').cast(BooleanType()))
#         read_res = read_res.withColumn('possible_invoice_nbr', col('possible_invoice_nbr').cast(StringType()))

#         for keyword in invoice_num_keywords:
#             read_res = read_res.withColumn('idx', col('content').lower().indexOf(keyword))
#             read_res = read_res.withColumn('idx', col('idx') + 1)
#             read_res = read_res.withColumn('potential_invoice_num', col('content').substr(col('idx'), 25))
#             read_res = read_res.withColumn('potential_invoice_num', col('potential_invoice_num').substr(1, expr("IF(instr(potential_invoice_num, 'date') > 0, instr(potential_invoice_num, 'date')-1, length(potential_invoice_num))")))

#             read_res = read_res.withColumn('text_list', expr("split(potential_invoice_num, ' ')"))
#             read_res = read_res.withColumn('possible_invoice_nbr', expr("transform(text_list, segment -> IF(REGEXP_REPLACE(segment, '\\D', '') != '', IF({0}, segment, NULL), NULL))".format(invalid_invoice_nbr_udf)))

#             read_res = read_res.withColumn('invalid_invoice_nbr', col('invalid_invoice_nbr') | col('possible_invoice_nbr').isNotNull())
#             read_res = read_res.withColumn('possible_invoice_nbr', expr("filter(possible_invoice_nbr, segment -> segment IS NOT NULL)[0]"))

#         return read_res

In [ ]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, lower, split, udf
from pyspark.sql.types import BooleanType, StringType

spark = SparkSession.builder.getOrCreate()
spark.udf.register("if_repeat_num_udf", if_repeat_num, BooleanType())
spark.udf.register("if_single_num_udf", if_single_num, BooleanType())
spark.udf.register("if_consecutive_num_udf", if_consecutive_num, BooleanType())

def invoice_num_check(read_res):
    if_single_num_udf = udf(if_single_num, BooleanType())
    if_repeat_num_udf = udf(if_repeat_num, BooleanType())
    if_consecutive_num_udf = udf(if_consecutive_num, BooleanType())

    #invoice_num_keywords = ['invoice number', 'invoice #', 'invoice num', 'invoice#']
    invoice_num_keywords = ['invoice num']
    read_res = read_res.withColumn('invalid_invoice_nbr', lit(False))
    read_res = read_res.withColumn('possible_invoice_nbr', lit(None))

    for keyword in invoice_num_keywords:
       # read_res = read_res.withColumn('idx', lower(col('content')).indexOf(keyword) + 1)
        read_res = read_res.withColumn('text', lower(col('content')))
        #read_res = read_res.withColumn('idx', col('text').instr(keyword))
        #read_res = read_res.withColumn('idx', col('idx') + 1)
        read_res = read_res.withColumn('keyword',lit(keyword))
        read_res = read_res.withColumn('idx', expr("INSTR(text, '{0}') - 1".format(keyword)))
        read_res = read_res.withColumn('potential_invoice_num', expr("substring(text, idx, 25)"))
        read_res = read_res.withColumn('potential_invoice_num', expr("substring(potential_invoice_num, 1, IF(instr(potential_invoice_num, 'date') > 0, instr(potential_invoice_num, 'date')-1, length(potential_invoice_num)))"))

        read_res = read_res.withColumn('text_list', split(col('potential_invoice_num'), ' '))
        read_res = read_res.withColumn('possible_invoice_nbr', expr("""
            TRANSFORM(text_list, segment ->
                IF(
                    REGEXP_REPLACE(segment, '\\D', '') != '' AND
                    (segment RLIKE '\\d{4}' OR segment RLIKE '[1-9]+\\\\1\\\\1\\\\1+'),
                    segment,
                    NULL
                )
            )
        """))
        # read_res = read_res.withColumn('possible_invoice_nbr', expr("""
        # TRANSFORM(text_list, segment ->
        #     IF(REGEXP_REPLACE(segment, '\\D', '') != '',
        #         IF(
        #             if_single_num_udf(segment) OR
        #             if_repeat_num_udf(segment) OR
        #             if_consecutive_num_udf(segment),
        #             segment,
        #             NULL
        #         ),
        #         NULL
        #     )
        # )
        # """))#.format(if_single_num_udf(col('segment')), if_repeat_num_udf(col('segment')), if_consecutive_num_udf(col('segment')))))

        #read_res = read_res.withColumn('possible_invoice_nbr', expr("transform(text_list, segment -> IF(REGEXP_REPLACE(segment, '\\D', '') != '', IF({0} OR {1} OR {2}, segment, NULL), NULL))".format(if_single_num_udf(col('segment')), if_repeat_num_udf(col('segment')), if_consecutive_num_udf(col('segment')))))

        read_res = read_res.withColumn('invalid_invoice_nbr', col('invalid_invoice_nbr') | col('possible_invoice_nbr').isNotNull())
        read_res = read_res.withColumn('possible_invoice_nbr', expr("filter(possible_invoice_nbr, segment -> segment IS NOT NULL)[0]"))
        #read_res = read_res.drop(*('text','idx'))

    return read_res


# address_check

In [ ]:
def address_check(read_res):
        def if_valid_address(address_text):
            regex_addr_validated = r'[a-zA-Z](\d|i|o|s)[a-zA-Z]\s?(\d|i|o|s)[a-zA-Z](\d|i|o)\s'
            road_name = ['avenue','ave','boulevard','blvd','circle','cir','court','ct','expressway','expy','freeway','fwy','lane','ln','parkway','pky','road','rd','square','sq','street','st','driveway','dr','drive','highway','hwy']
            province_name = ['albert','ab','british columbia','bc','manitoba','mb','new brunswick','nb','newfoundland and labrador', 'nl', 'northwest territories', 'nt', 'nova scotia', 'ns', 'nunavut', 'nu','ontario','on', 'prince edward island', 'pe', 'quebec', 'qc', 'saskatchewan', 'sk', 'yukon', 'yt']
            exception_text = ['manulife','image']

            add_regex = re.compile(regex_addr_validated)
            search_res = re.search(add_regex, address_text)

            if search_res and any(substring in address_text for substring in road_name) and any(substring in address_text for substring in province_name):
                return True
            elif not any(substring in address_text for substring in exception_text):
                return False
            else:
                return True

        if_valid_address_udf = udf(if_valid_address, BooleanType())

        read_res = read_res.withColumn('has_invalid_addr', col('has_invalid_addr').cast(BooleanType()))
        read_res = read_res.withColumn('verified_address', col('verified_address').cast(StringType()))

        invoice_num_keywords = ['address']
        for keyword in invoice_num_keywords:
            read_res = read_res.withColumn('idx', col('content').lower().indexOf(keyword))
            read_res = read_res.withColumn('idx', col('idx') + 1)
            read_res = read_res.withColumn('potential_add_text', col('content').substr(col('idx'), 80))
            read_res = read_res.withColumn('potential_add_text', expr("IF(instr(potential_add_text, 'email address') > 0, substring(potential_add_text, 1, instr(potential_add_text, 'email address')-1), potential_add_text)"))

            read_res = read_res.withColumn('has_invalid_addr', if_valid_address_udf(col('potential_add_text')) | col('has_invalid_addr'))

        p = re.compile(r'\s[a-zA-Z](\d|i|o)[a-zA-Z]\s?(\d|i|o)[a-zA-Z](\d|i|o)\s')
        read_res = read_res.withColumn('text_matches', expr("transform_all(content, x -> regexp_extract_all(x, '\\s[a-zA-Z](\\d|i|o)[a-zA-Z]\\s?(\\d|i|o)[a-zA-Z](\\d|i|o)\\s', 0))"))
        read_res = read_res.withColumn('potential_add_text', expr("explode_outer(flatten(text_matches))"))
        read_res = read_res.withColumn('has_invalid_addr', if_valid_address_udf(col('potential_add_text')) | col('has_invalid_addr'))

        return read_res

# Test

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, lit, split, expr, when
from pyspark.sql.types import BooleanType

def invoice_num_check(read_res):
    invoice_num_keywords = ['invoice number', 'invoice #', 'invoice num', 'invoice#']
    read_res = read_res.withColumn('invalid_invoice_nbr', lit(False))
    read_res = read_res.withColumn('possible_invoice_nbr', lit(None))

    for keyword in invoice_num_keywords:
        read_res = read_res.withColumn('text', lower(col('content')))
        read_res = read_res.withColumn('idx', expr("INSTR(text, '{0}') - 1".format(keyword)))
       # read_res = read_res.withColumn('potential_invoice_num', col('text').substr(col('idx'), 25))
        read_res = read_res.withColumn('potential_invoice_num', expr("substring(text, idx, 25)"))
        #read_res = read_res.withColumn('potential_invoice_num', col('potential_invoice_num').substr(1, expr("IF(instr(potential_invoice_num, 'date') > 0, instr(potential_invoice_num, 'date')-1, length(potential_invoice_num))")))
        read_res = read_res.withColumn('potential_invoice_num', expr("substring(potential_invoice_num, 1, IF(instr(potential_invoice_num, 'date') > 0, instr(potential_invoice_num, 'date')-1, length(potential_invoice_num)))"))

        read_res = read_res.withColumn('text_list', split(col('potential_invoice_num'), ' '))
        read_res = read_res.withColumn('possible_invoice_nbr', expr("""
            TRANSFORM(text_list, segment ->
                IF(
                    REGEXP_REPLACE(segment, '\\D', '') != '' AND
                    (segment RLIKE '\\d{4}' OR segment RLIKE '[1-9]+\\\\1\\\\1\\\\1+'),
                    segment,
                    NULL
                )
            )
        """))

        read_res = read_res.withColumn('invalid_invoice_nbr', col('invalid_invoice_nbr') | col('possible_invoice_nbr').isNotNull())
        read_res = read_res.withColumn('possible_invoice_nbr', expr("filter(possible_invoice_nbr, segment -> segment IS NOT NULL)[0]"))

    return read_res

# Example usage
spark = SparkSession.builder.getOrCreate()
data = [("Hello, how are you?",), ("I'm doing well, thank you!",)]
df = spark.createDataFrame(data, ["content"])
result = invoice_num_check(df)
result.show()


+--------------------+-------------------+--------------------+--------------------+---+---------------------+---------+
|             content|invalid_invoice_nbr|possible_invoice_nbr|                text|idx|potential_invoice_num|text_list|
+--------------------+-------------------+--------------------+--------------------+---+---------------------+---------+
| Hello, how are you?|               true|                null| hello, how are you?| -1|                    ?|      [?]|
|I'm doing well, t...|               true|                null|i'm doing well, t...| -1|                    !|      [!]|
+--------------------+-------------------+--------------------+--------------------+---+---------------------+---------+



In [ ]:
df.dtypes

[('file_name', 'string'),
 ('content', 'string'),
 ('page_label', 'string'),
 ('hand', 'double')]

In [ ]:
df.show()

+---------+--------------------+----------+-----+
|file_name|             content|page_label| hand|
+---------+--------------------+----------+-----+
|test1.jpg|Payment made to c...|      para|30.32|
|test2.jpg|Transaction was d...|       mil| 10.2|
|test3.jpg|User with registe...|       bil| 27.9|
+---------+--------------------+----------+-----+



In [ ]:
def invoice_num_check(read_res):
  read_res = read_res.withColumn("invalid_invoice_nbr", lit(False))
  read_res = read_res.withColumn("possible_invoice_nbr", lit(None))

  invoice_num_keywords = ['invoice number', 'invoice #', 'invoice num', 'invoice#']
  invoice = "None"
  def extract_invoice(text):
    text = text.lower()

    for keyword in invoice_num_keywords:
        idx = text.find(keyword)
        if idx == -1:
          continue
        potential_invoice_num = text[idx: idx+25] if idx+25 < len(text) else text[idx:]
        if potential_invoice_num.find('date')>-1:
            potential_invoice_num = potential_invoice_num[:potential_invoice_num.find('date')]

        text_list = potential_invoice_num.split(' ')
        for segment in text_list:
            if any(c.isdigit() for c in segment):
                if if_single_num(segment):
                    return segment
                elif if_repeat_num(segment):
                    return segment
                elif if_consecutive_num(segment):
                    return segment
                else:
                    #read_res.loc[row[0], 'possible_invoice_nbr'] = segment
                  return segment
    return invoice

  extract_invoice_udf = udf(extract_invoice, StringType())
  read_res = read_res.withColumn("possible_invoice_nbr", extract_invoice_udf(col("content")))
  return read_res


In [ ]:
res_df=invoice_num_check(df)

In [ ]:
res_df.show()

+---------+--------------------+----------+-----+-------------------+--------------------+
|file_name|             content|page_label| hand|invalid_invoice_nbr|possible_invoice_nbr|
+---------+--------------------+----------+-----+-------------------+--------------------+
|test1.jpg|Payment made to c...|      para|30.32|              false|               36241|
|test2.jpg|Transaction was d...|       mil| 10.2|              false|       invoice#12534|
|test3.jpg|User with registe...|       bil| 27.9|              false|                None|
+---------+--------------------+----------+-----+-------------------+--------------------+



In [ ]:
res_df.dtypes

[('file_name', 'string'),
 ('content', 'string'),
 ('page_label', 'string'),
 ('hand', 'double'),
 ('invalid_invoice_nbr', 'boolean'),
 ('possible_invoice_nbr', 'string')]

In [ ]:
expected_data = [("test1.jpg", "Payment made to cashier with cash of 2000.00 ", "para",30.32,2000.0),
                        ("test2.jpg", "Transaction was done using cash in costco", "mil",10.2),
                            ("test3.jpg","User with register no: 58290 used debit card for payment","bil",27.9)]
headers_exp = ("file_name", "content", "page_label","hand","payment_amount")
expected_df = sc.createDataFrame(expected_data, headers_exp)

In [ ]:
expected_df.dtypes

[('file_name', 'string'),
 ('content', 'string'),
 ('page_label', 'string'),
 ('hand', 'double'),
 ('payment_amount', 'double')]

In [1]:
!unzip -q ../content/fd_spark.zip -d ../content/

In [1]:
import os
import json
import lxml
from bs4 import BeautifulSoup as bs
from pathlib import Path
import sys
import json
import csv
import pandas as pd
# from azure.core.credentials import AzureKeyCredential
# from azure.ai.formrecognizer import DocumentAnalysisClient
# from azureml.core import Workspace, Dataset,Datastore
# from azureml.data.datapath import DataPath
import ast
import re
import numpy as np
from pathlib import Path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
path_to_module = '../content/Fraud_detection/'
sys.path.append(path_to_module)


In [2]:
from computer_version import computer_version_rules
from form_recognizer import doc_read
from rules import fraud_document_analytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
input_path = "../content/Fraud_detection/test/"
local_output_path = '../content/output/'

In [4]:
cv = computer_version_rules(input_path,local_output_path)

In [5]:
fr = doc_read(input_path,local_output_path)

In [6]:
date_name = 'Test'
batch_name = '111'
input_path =  "../content/Fraud_detection/test/"
files_with_stamp = cv.detect_stamp(date_name,batch_name,input_path)
test2  = fr.read_document(date_name,batch_name,input_path)
test2  = fr.sort_document(batch_name, test2)
# test2  = fr.parse_metadata(batch_name, input_path,local_output_path, test2)
test2 = fr.handwritten_per(batch_name,test2)
test2 = fr.check_stamp(test2,files_with_stamp)

i am in
0
1
140748
 In read doc
 After document_analysis_client doc
AC_Beauty_and_Wellness.pdf
Advantege_Medical_Center.pdf
Advanced_Health_Rehabilitation.pdf
Read 3 images' text in batch 111
I am in sort_doc
Labelled batch 111
Calculating handwritten percentage of files in batch 111


In [8]:
files_with_stamp

[]

In [9]:
test2

,file_name,content,word_confidence,styles,page_label,handwritten_percentage,has_stamp
0,Test_111_AC_Beauty_and_Wellness.pdf,Provider Not Listed Claim Confirmation # Inter...,"{'Provider': 1, 'Not': 1, 'Listed': 1, 'Claim'...",None,other_doc,0,False
1,Test_111_Advantege_Medical_Center.pdf,,{},None,other_doc,0,False
2,Test_111_Advanced_Health_Rehabilitation.pdf,CONFIDENTIAL CONFIDENTIAL,{'CONFIDENTIAL': 1},None,other_doc,0,False


In [10]:
from rules import fraud_document_analytics

In [11]:
rs = fraud_document_analytics()

In [13]:
test2 = rs.paid_by_cash(test2)
test2 = rs.has_dollar_symbol(test2)
test2 = rs.handwritten_check(test2,0.3)
test2 = rs.invoice_num_check(test2)
test2 = rs.get_payment(test2)
test2 = rs.address_check(test2)
test2 = rs.get_reg_num(test2)

In [14]:
test2

,file_name,content,word_confidence,styles,page_label,handwritten_percentage,has_stamp,paid_by_cash,no_dollar_symbol,above_handwritten_threshold,invalid_invoice_nbr,possible_invoice_nbr,payment_amount,has_invalid_addr,verified_address,register_num
0,Test_111_AC_Beauty_and_Wellness.pdf,Provider Not Listed Claim Confirmation # Inter...,"{'Provider': 1, 'Not': 1, 'Listed': 1, 'Claim'...",None,other_doc,0,False,False,False,False,False,None,450.0,False,None,None
1,Test_111_Advantege_Medical_Center.pdf,,{},None,other_doc,0,False,False,True,False,False,None,NaN,False,None,None
2,Test_111_Advanced_Health_Rehabilitation.pdf,CONFIDENTIAL CONFIDENTIAL,{'CONFIDENTIAL': 1},None,other_doc,0,False,False,True,False,False,None,NaN,False,None,None


In [17]:
test2.to_csv("output.csv")

In [16]:
test2["content"]

0    Provider Not Listed Claim Confirmation # Inter...
1                                                     
2                   CONFIDENTIAL   
      CONFIDENTIAL
Name: content, dtype: object

# PYSPARK Code

In [1]:
#!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
#from azure.storage.blob import BlobServiceClient
import os
from pyspark.sql.functions import col, when

#spark = SparkSession.builder.getOrCreate()

In [3]:
input_path = "../content/fd_spark/test/"
local_output_path = '../content/output/'

In [4]:
import os
from pathlib import Path
import sys
import json
import csv
import pandas as pd
import re
import numpy as np
from pathlib import Path
path_to_module = '../content/fd_spark/'
sys.path.append(path_to_module)
from computer_version import ComputerVersionRules
from Form_Recognizer import doc_read
from rules import fraud_document_analytics

In [5]:
cv = ComputerVersionRules(input_path,local_output_path)

In [6]:
fr = doc_read()

In [7]:
date_name = 'Test'
batch_name = '111'
batch_path =  "../content/fd_spark/test/"
files_with_stamp = cv.detect_stamp(date_name,batch_name,input_path)
files_with_stamp = files_with_stamp.collect()
test2  = fr.read_document(date_name,batch_name,batch_path)
#test2  = fr.sort_document(batch_name, test2)
# test2  = fr.parse_metadata(batch_name, input_path,local_output_path, test2)
test2 = fr.handwritten_per(batch_name,test2)
test2 = fr.check_stamp(test2,files_with_stamp)

In [8]:
test2.show()

+--------------------+--------------------+--------------------+------+----------------------+---------+
|           file_name|             content|     word_confidence|styles|handwritten_percentage|has_stamp|
+--------------------+--------------------+--------------------+------+----------------------+---------+
|Test_111_AC Beaut...|Provider Not List...|{Apr=1, 02=1, Mem...|  null|                  null|    false|
|Test_111_Advanced...|CONFIDENTIAL   \f...|    {CONFIDENTIAL=1}|  null|                  null|    false|
|Test_111_Advanteg...|                    |                  {}|  null|                  null|    false|
+--------------------+--------------------+--------------------+------+----------------------+---------+



In [9]:
rs = fraud_document_analytics()

In [10]:
test2 = rs.paid_by_cash(test2)
test2 = rs.has_dollar_symbol(test2)
test2 = rs.handwritten_check(test2,0.3)
test2 = rs.invoice_num_check(test2)
test2 = rs.get_payment(test2)
test2 = rs.address_check(test2)
test2 = rs.get_reg_num(test2)

In [11]:
test2.show()

+--------------------+--------------------+--------------------+------+----------------------+---------+------------+----------------+---------------------------+-------------------+--------------------+--------------+----------------+---------------+
|           file_name|             content|     word_confidence|styles|handwritten_percentage|has_stamp|paid_by_cash|no_dollar_symbol|above_handwritten_threshold|invalid_invoice_nbr|possible_invoice_nbr|payment_amount|has_invalid_addr|   register_num|
+--------------------+--------------------+--------------------+------+----------------------+---------+------------+----------------+---------------------------+-------------------+--------------------+--------------+----------------+---------------+
|Test_111_AC Beaut...|provider not list...|{Apr=1, 02=1, Mem...|  null|                  null|    false|       false|           false|                       null|              false|                None|         450.0|           false|No Regist

In [13]:
output_path = "/content/output_spark.csv"
test2.repartition(1).write.csv(output_path, header=True, mode="overwrite")